In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pickle
from time import sleep

In [2]:
INSTAGRAM_HOST_NAME = 'https://www.instagram.com/'

In [3]:
def get_tag_count(tag):
    try:
        url = INSTAGRAM_HOST_NAME+ 'explore/tags/' +  tag
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        script = soup.findAll("script", text=re.compile("window._sharedData"))[0].text
        json_str = script.replace('window._sharedData = ', '').replace(';', '')
        data = json.loads(json_str)
        return data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['count']
    except Exception as e:
        print("CANT GET DATA : tag_name = %s; %s;" % (tag, e))
        return -1

In [4]:
def get_username_by_shortcode(shortcode):
    try:
        url = INSTAGRAM_HOST_NAME + 'p/' + shortcode
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        script = soup.findAll("script", text=re.compile("window._sharedData"))[0].text
        json_str = script.replace('window._sharedData = ', '').replace(';', '')
        data = json.loads(json_str)
        return data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['owner']['username']
    except Exception as e:
        print("CANT GET DATA : shortcode = %s; %s;" % (shortcode, e))

In [5]:
def get_followed_by_username(username):
    try:
        url = INSTAGRAM_HOST_NAME + username
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        script = soup.findAll("script", text=re.compile("window._sharedData"))[0].text
        json_str = script.replace('window._sharedData = ', '').replace(';', '')
        data = json.loads(json_str)
        return data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_followed_by']['count']
    except Exception as e:
        print("CANT GET DATA : username = %s; %s;" % (username, e))

In [6]:
def caption2tags(caption):
    candidates = caption.split(' ')
    tags = []
    for tag in candidates:
        if tag[0] == '#':
            count = get_tag_count(tag[1:])
            if count != -1:
                tags.append({tag[1:]: count})
    return tags

In [7]:
def get_data_by_tag(tag):
    posts = []
    try:
        url = INSTAGRAM_HOST_NAME+ 'explore/tags/' +  tag
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        script = soup.findAll("script", text=re.compile("window._sharedData"))[0].text
        json_str = script.replace('window._sharedData = ', '').replace(';', '')
        data = json.loads(json_str)
        for node in data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
            post = {}
            post['shortcode'] = node['node']['shortcode']
            username = get_username_by_shortcode(post['shortcode'])
            post['followered'] = get_followed_by_username(username)
            post['tag'] = caption2tags(node['node']['edge_media_to_caption']['edges'][0]['node']['text'])
            post['image'] = node['node']['display_url']
            post['like'] = node['node']['edge_liked_by']['count']

            posts.append(post)
            sleep(1)
        return posts
    except Exception as e:
        print("CANT GET DATA : tag_name = %s; %s;" % (tag, e))

In [8]:
tag = 'nagaokatenmangu'

In [ ]:
posts = get_data_by_tag(tag)

In [ ]:
posts

In [ ]:
# with open("/root/userspace/finalwork/data/posts.pcl", "wb") as pf:
with open("./data/posts.pcl", "wb") as pf:
    pickle.dump(posts, pf)